# Transit Corridor Improvements

## Criteria
* Transit routes that intersect the SHN (at least 35% of the route is within a 0.5 mile buffer of the SHN)
* average route speed is below **12 mph**  **AND**
* **10%+ of its trips are competitive** against a car 
    * The car travels the same path as the bus (same stops along the way).
    * A bus trip is competitive if it is within 1.5x the car's travel time (bus can take 50% longer).
    * Across all the trips taken by that route, if **at least 10% trips** are competitive, the route is deemed competitive. 
    * Avoid selecting 0% trips competitive, which gets at circulator/loop type routes

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca 
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

from calitp_data_analysis import calitp_color_palette as cp
from shared_utils import portfolio_utils
from bus_service_utils import better_bus_utils

catalog = intake.open_catalog("../bus_service_increase/*.yml")

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple  
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
]

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
speed_dict = {"mean_speed_mph": 12}
trip_dict = {"pct_trips_competitive": 0.10}

gdf = better_bus_utils.select_transit_routes_corridor_improvements(
    speed_dict = speed_dict,
    trip_dict = trip_dict
)

In [ ]:
def subset_to_district(gdf: gpd.GeoDataFrame, district: str) -> gpd.GeoDataFrame: 
    keep_cols = ["calitp_itp_id", "route_id", 
            "mean_speed_mph", "pct_trips_competitive",
            "category", "geometry"]
    
    df = (gdf[gdf.caltrans_district == district]
          .reset_index(drop=True)
          [keep_cols]
         )
    
    df2 = df.assign(
        mean_speed_mph = df.mean_speed_mph.round(2),
        pct_trips_competitive = df.pct_trips_competitive.round(3)
    )
               
    return df2

In [ ]:
TILES = "CartoDB Positron" 

def make_map(gdf: gpd.GeoDataFrame, district: str):        
    m = gdf.explore("mean_speed_mph", categorical = False, 
                    cmap = ZERO_THIRTY_COLORSCALE, tiles = TILES)
    
    display(m)
    
    m2 = gdf.explore("route_id", categorical = True, 
                    cmap = "tab20", tiles = TILES)
    
    display(m2)
    
    gdf = gdf.sort_values(["mean_speed_mph", "pct_trips_competitive"], 
                         ascending=[True, True]).reset_index(drop=True)
    
    df_style = portfolio_utils.style_table(
        gdf,
        drop_cols = ["geometry", "category"],
        rename_cols = {"mean_speed_mph": "Avg Speed (mph)", 
                       "pct_trips_competitive": "% trips competitive"
                      },
        one_decimal_cols = ['Avg Speed (mph)'],
        custom_format_cols = {"{:,.1%}": ['% trips competitive']},
        display_table = True
    )


## District Maps

In [ ]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(HTML(f"<h3> District {d}</h3>"))

    subset = subset_to_district(gdf, d)

    if len(subset) > 0:
        make_map(subset, d)
    else:
        print("No routes meet this criteria")